# Marginal Emissions
## Demonstration on using the marginal emissions feature of NEMED
Using the package <code>nemed</code> to calculate hourly marginal emissions over a historical period.

### Data Preparation
#### Import Packages

In [ ]:
import nemed

# To generate plots shown
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
# Open plot in browser (optional)
import plotly.io as pio
pio.renderers.default = "notebook"

#### Define Parameters
```{tip}
Current Implementation will only load data based on `start_time` day and `end_time` day with the data series commencing
at 04:05:00 and ending at 04:00:00
```

In [ ]:
cache = "E:/TEMPCACHE_LX/"
start_time = "2022/07/01 00:00:00" 
end_time = "2022/07/02 00:00:00"
filter_units = None 
filter_regions = None

#### Get data using NEMED

In [ ]:
result = nemed.get_marginal_emissions(start_time, end_time, cache, redownload_xml=False)

#### Aggregate data
By calling <code>aggregate_marginal_data_by</code> you can quickly organise data into a suitable aggregation.
Some examples are:

In [ ]:
from nemed.process import *
#test = result.copy()
test = result.copy().set_index('PeriodID')
type(test.index[0])
aggregate_marginal_data_by(test, by="interval", maintain_dates=False, agg="mean")

In [ ]:
a = pd.Series([1,2,3])
isinstance(a, pd.Series)

In [ ]:
pd.DataFrame(a)

In [ ]:
test.index.name = 'a'

In [ ]:
if (not isinstance(test.index.values[0], (datetime, pd.Timestamp))) and (test.index.name != 'PeriodID'):
    raise ValueError("`data` parsed must have datetime type as index with name 'PeriodID'")


In [ ]:
test.index.name

In [ ]:
pd.Timestamp

In [ ]:
result.set_index('PeriodID',inplace=True)
result.index.name

In [ ]:
inpdf = test[['PeriodID','RegionID','CO2E_EMISSIONS_FACTOR']].drop_duplicates()
inpdf.set_index('PeriodID',inplace=True)
aggregate_marginal_data_by(inpdf, by="day", agg="mean")

In [ ]:
inpdf.index = [datetime(year=1000, month=1, day=1, hour=inpdf.index[i].hour, minute=inpdf.index[i].minute) for i in range(len(inpdf.index))]

In [ ]:
inpdf

In [ ]:
aggregate_marginal_data_by(inpdf, by="hour", agg="mean")

In [ ]:
# Filter select region
region = 'SA1'
reg_sum = result[result['RegionID'] == region]
reg_sum = reg_sum.drop_duplicates(subset=['PeriodID','CO2E_EMISSIONS_FACTOR','tech_name'])
reg_sum = reg_sum.reset_index(drop=True)[['PeriodID','RegionID','Dispatch Type','tech_name','CO2E_EMISSIONS_FACTOR','Date','Hour']]

# if tech name matches and em factor matches for same time, drop duplicates

In [ ]:
reg_sum.set_index('PeriodID',inplace=True)

In [ ]:
reg_sum[['CO2E_EMISSIONS_FACTOR']].resample('30min', label='right', origin='end').agg(np.min)

In [ ]:
from nemed.process import *

In [ ]:
aggregate_marginal_data_by(result, by="halfhour", agg="mean")

In [ ]:
region = 'SA1'
import numpy as np
from datetime import datetime

In [ ]:
#result.set_index('PeriodID',inplace=True)
data=result
result = data.groupby(by=[data.index.year,data.index.month,data.index.day,data.index.hour]).sum()

# result["reconstr_time"] = None
# for row in result.index:
#     result.loc[row, "reconstr_time"] = datetime(year=row[0], month=row[1], day=row[2], hour=row[3])
# result.set_index("reconstr_time", inplace=True)
# result.index.name = "Time"

In [ ]:
result

In [ ]:
reg_sum[reg_sum.duplicated()]

In [ ]:
hourly_agg = reg_sum.groupby(by=['Hour']).mean()
hourly_agg.reset_index(inplace=True)
#hourly_agg['Timestamp'] = [str(hourly_agg['Date'][i]) + " " + str(hourly_agg['Hour'][i]) for i in range(len(hourly_agg))]
hourly_agg

#### Setup Plotting
##### Marginal Emissions Intensity Plot Format

In [ ]:
def plot_marginal_emissions_intensity(region, color_idx):

    fig = go.Figure()

    # fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
    #     specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]])
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Marginal Emissions Intensity<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Emissions Intensity (tCO2-e/MWh)")
    fig.update_xaxes(title_text="Date (Day)")

    # fig.add_trace(go.Scatter(x=reg_sum['PeriodID'],y=reg_sum['CO2E_EMISSIONS_FACTOR'],\
    #     name="NEMED Marginal Intensity",mode="lines+markers",line_color=colors[color_idx]))

    fig.add_trace(go.Scatter(x=hourly_agg['Hour'],y=hourly_agg['CO2E_EMISSIONS_FACTOR'],\
        name="NEMED Marginal Intensity",mode="lines+markers",line_color=colors[color_idx]))

    # fig.add_trace(go.Scatter(x=energy.index,y=error_pct_en,\
    #     name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

In [ ]:
plot_marginal_emissions_intensity('NSW1',3)

##### Total Emissions Plot Format

In [ ]:
def plot_emissions(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_total_emissions_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Total Emissions<br><sub>nsw Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Emissions (tCO2-e)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['TOTAL_EMISSIONS'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index,y=emissions_tot[region].values,\
        name="nswED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

##### Emissions Intensity (Average Emissions) Plot Format

In [ ]:
def plot_intensity(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_intensity_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Average Emissions (Emissions Intensity)<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Emissions (tCO2-e/MWh)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['CO2E_INTENSITY_INDEX'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index,y=emissions_avg[region].values,\
        name="NEMED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines", line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

### Results - Comparison to AEMO

#### NEM

In [ ]:
region = 'NEM'
plot_energy(region, color_idx=0)
plot_emissions(region, color_idx=0)
plot_intensity(region, color_idx=0)

```{include} charts_ex1/energy_nem.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_nem.html)

```{include} charts_ex1/emissions_nem.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_nem.html)

```{include} charts_ex1/intensity_nem.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_nem.html)

#### NSW

In [ ]:
region = 'NSW1'
plot_energy(region, color_idx=1)
plot_emissions(region, color_idx=1)
plot_intensity(region, color_idx=1)

```{include} charts_ex1/energy_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_nsw.html)

```{include} charts_ex1/emissions_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_nsw.html)

```{include} charts_ex1/intensity_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_nsw.html)

#### VIC

In [ ]:
region = 'VIC1'
plot_energy(region, color_idx=3)
plot_emissions(region, color_idx=3)
plot_intensity(region, color_idx=3)

```{include} charts_ex1/energy_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_vic.html)

```{include} charts_ex1/emissions_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_vic.html)

```{include} charts_ex1/intensity_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_vic.html)

#### QLD

In [ ]:
region = 'QLD1'
plot_energy(region, color_idx=2)
plot_emissions(region, color_idx=2)
plot_intensity(region, color_idx=2)

```{include} charts_ex1/energy_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_qld.html)

```{include} charts_ex1/emissions_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_qld.html)

```{include} charts_ex1/intensity_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_qld.html)

#### SA

In [ ]:
region = 'SA1'
plot_energy(region, color_idx=4)
plot_emissions(region, color_idx=4)
plot_intensity(region, color_idx=4)

```{include} charts_ex1/energy_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_sa.html)

```{include} charts_ex1/emissions_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_sa.html)

```{include} charts_ex1/intensity_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_sa.html)

#### TAS

In [ ]:
region = 'TAS1'
plot_energy(region, color_idx=5)
plot_emissions(region, color_idx=5)
plot_intensity(region, color_idx=5)

```{include} charts_ex1/energy_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_tas.html)

```{include} charts_ex1/emissions_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_tas.html)

```{include} charts_ex1/intensity_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_tas.html)